# Reading events from PPs and adding Tracking

Currently the code will read the PPs from the pp_info.csv file, then it will find relevant events from the event file that match. After that it will attempt to get tracking data that is relevant. 

First it will figure out the time of the event relevant to the frame time of the tracking info (use the videoShotsInfo files). Then it will assign each file with the tracking data between it and then next event. In order to avoid events that are close together not getting any tracking at all I assign 15 frames back and 15 forward to each event just in case (1 second extra tracking), this also accounts for the limited granularity in the event timing (which is 1 sec vs 30 fps for tracking)

Tracking data is corrected as per the tweet we found, but not doing anything about gaps or about "player 100s"

I am not saving it at the moment because it is nested and doesn't take long to run.

In [1]:
from read_and_join_events_tracking import *
import os

# print(os.path.join('../','pxp_womens_oly_2022_v2.csv'))

relevant_events = get_pp_tracking(tracking_dir='./', pbp_dir = '../')


/mnt/c/Users/thepi/OneDrive/Documents/Python/Hockey/Big-Data-Cup-2022-Private/TrackingData/read_and_join_events_tracking.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  power_play_info['game_name'].loc[power_play_info['game_name']=='2022-02-14 USA at Finland'] = '2022-02-14 Finland at USA'


Tracking file missing:
./2022-02-08 Canada at USA/2022-02-08 Canada at USA P2 PP4.csv
Tracking file missing:
./2022-02-12 Switzerland at ROC/2022-02-12 Switzerland at ROC P3 PP4.csv
Tracking file missing:
./2022-02-14 Switzerland at Canada/2022-02-14 Switzerland at Canada P2 PP3.csv
Tracking file missing:
./2022-02-14 Finland at USA/2022-02-14 Finland at USA P2 PP2.csv
Tracking file missing:
./2022-02-16 Switzerland at Finland/2022-02-16 Switzerland at Finland P1 PP3.csv


# Example of the format of the output:

A dataframe containing each PP, with a column named "relevant_events" which contains a DF with event data

In [6]:
relevant_events.head()

,game_name,penalty_number,start_video_clock_seconds,end_video_clock_seconds,start_period,end_period,start_game_clock_seconds,end_game_clock_seconds,relevant_events
0,2022-02-08 Canada at USA,1,1278,1314,1,1,386,350,game_date season_year ...
1,2022-02-08 Canada at USA,2,1624,1744,1,1,216,96,game_date season_year ...
2,2022-02-08 Canada at USA,3,409,562,2,2,991,871,game_date season_year ...
3,2022-02-08 Canada at USA,4,898,898,2,2,613,613,None
4,2022-02-08 Canada at USA,5,1690,1927,2,2,272,152,game_date season_year ...


Example of one DF of event data, which contains a column of tracking data called "tracks", which itself is a DF

In [3]:
relevant_events.loc[0,'relevant_events'].head()

,game_date,season_year,team_name,opp_team_name,venue,period,clock_seconds,situation_type,goals_for,goals_against,...,event_type,player_name_2,x_coord_2,y_coord_2,event_detail_1,event_detail_2,event_detail_3,Player_1_num,Player_2_num,tracks
0,8/2/2022,2021,Olympic (Women) - United States,Olympic (Women) - Canada,home,1,386,5 on 5,0,0,...,Cross-checking,Sarah Fillier,NaN,NaN,NaN,NaN,NaN,4,10.0,None
1,8/2/2022,2021,Olympic (Women) - United States,Olympic (Women) - Canada,home,1,386,4 on 5,0,0,...,Backhand,Marie-Philip Poulin,NaN,NaN,NaN,NaN,NaN,25,29.0,None
2,8/2/2022,2021,Olympic (Women) - United States,Olympic (Women) - Canada,home,1,383,4 on 5,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,None
3,8/2/2022,2021,Olympic (Women) - United States,Olympic (Women) - Canada,home,1,382,4 on 5,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,None
4,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,380,5 on 4,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,None


Finally, an example of one DF of tracking data

In [7]:
relevant_events.loc[0,'relevant_events'].tracks.iloc[6]

,frame_id,period,track_id,team_id,team_name,jersey_number,x_ft,y_ft,vel_x,vel_y
98,315,1,1,Light,Canada,6,166.412354,54.525812,5.482729,8.911096
99,315,1,2,Dark,USA,2,183.106257,48.074749,-6.806727,4.755684
100,315,1,3,Dark,USA,37,187.198140,51.961487,1.554065,0.914565
101,315,1,4,Light,Canada,11,195.635778,73.999070,2.143761,1.141240
102,315,1,5,Dark,USA,28,166.892412,46.917458,2.140943,8.115468
...,...,...,...,...,...,...,...,...,...,...
805,404,1,13,Light,Canada,100,155.659857,82.385167,14.886233,2.931767
806,404,1,16,Light,Canada,6,154.330091,64.552903,-8.442085,2.744008
807,404,1,18,Dark,USA,3,183.735250,55.444952,-0.549836,-2.666694
808,404,1,7,Dark,USA,5,173.488274,67.395426,-1.012685,3.871421


In [5]:

# tracks = relevant_events.loc[0,'relevant_events'].tracks.iloc[6].copy()
# tracks['vel_x'] = 0
# tracks['vel_y'] = 0
# tr_ids = tracks.track_id.unique()
# for tr_id in tr_ids:
#     idxs = tracks.track_id==tr_id
#     track = tracks.loc[idxs]
#     x_spd,y_spd = get_speed(track)
#     tracks.loc[idxs,'vel_x'] = x_spd
#     tracks.loc[idxs,'vel_y'] = y_spd
#     break
# print(tracks.head())